In [60]:
import requests
import email
from pathlib import Path
import json
from urllib.request import urlopen
import json
import plotly.express as px


In [ ]:

data = {
    "operationName": "AvailabilityCalendar",
    "variables": {
        "input": {
            "travelPurpose": 2,
            "pagenameDetails": {
                "countryCode": "za",
                "pagename": "brand-new-seapoint-apartment"
            },
            "searchConfig": {
                "searchConfigDate": {
                    "startDate": "2025-05-01",
                    "amountOfDays": 61
                },
                "nbAdults": 2,
                "nbRooms": 4,
                "nbChildren": 0,
                "childrenAges": []
            }
        }
    },
    "extensions": {},
    "query": "query AvailabilityCalendar($input: AvailabilityCalendarQueryInput!) {\n  availabilityCalendar(input: $input) {\n    ... on AvailabilityCalendarQueryResult {\n      hotelId\n      days {\n        available\n        avgPriceFormatted\n        avgPrice\n        checkin\n        minLengthOfStay\n        __typename\n      }\n      __typename\n    }\n    ... on AvailabilityCalendarQueryError {\n      message\n      __typename\n    }\n    __typename\n  }\n}\n"
}
r = requests.post("https://www.booking.com/dml/graphql?lang=en", json=data)
r.raise_for_status()
print(json.dumps(r.json(), indent=2))


{
  "data": {
    "availabilityCalendar": {
      "days": [
        {
          "avgPrice": 0.0,
          "minLengthOfStay": 0,
          "avgPriceFormatted": "\u00a30",
          "available": false,
          "checkin": "2025-10-25",
          "__typename": "AvailabilityCalendarDay"
        },
        {
          "available": false,
          "checkin": "2025-10-24",
          "__typename": "AvailabilityCalendarDay",
          "avgPrice": 0.0,
          "minLengthOfStay": 0,
          "avgPriceFormatted": "\u00a30"
        },
        {
          "avgPrice": 0.0,
          "avgPriceFormatted": "\u00a30",
          "minLengthOfStay": 0,
          "available": false,
          "checkin": "2025-10-27",
          "__typename": "AvailabilityCalendarDay"
        },
        {
          "available": false,
          "checkin": "2025-10-26",
          "__typename": "AvailabilityCalendarDay",
          "avgPrice": 0.0,
          "avgPriceFormatted": "\u00a30",
          "minLengthOfStay": 0
   

In [88]:
r = requests.get("https://www.booking.com/hotel/gr/seaside-gouves.html")
r.raise_for_status()    
with Path("response.html").open("w", encoding="utf-8") as f:
    f.write(r.text)